# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [5]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/anaconda3/envs/tf1/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/anaconda3/envs/tf1/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [10]:
input_text = "解释下乾卦是什么？"

In [11]:
response, history = base_model.chat(tokenizer, query=input_text)

In [12]:
print(response)

乾卦是《易经》中的第一卦，它的卦象是由六个阳爻夹一个阴爻构成，象征着天、强、积极、刚健等含义。乾卦的卦辞为：“元、亨、利、贞”，意味着“亨通、顺利、吉祥、正道”。

乾卦象征着阳刚之氣，万物创造者，领导者的象征。它告诉我们，阳刚之气是推动事物发展的重要力量，只有保持积极向上的心态和行动，才能达到成功。

乾卦的应用广泛，可以指导我们在生活和工作中，如何发挥阳刚之氣，如何处理各种问题和挑战，如何达到成功和幸福。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [13]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

周易中的讼卦（cording to the I Ching）是八卦之一，它的卦象是由两个阴爻夹一个阳爻构成，象征着决断、诉讼、争斗、疾病等象征。

讼卦的卦辞为：“天行健，君子以自强不息；君子有终，吉。”意味着“天行健行，君子应该自强不息；君子有恒心，吉祥”。

讼卦意味着争斗和诉讼，但也象征着决断和正义。它告诉我们，当遇到争斗和诉讼时，要坚定自己的信念，保持正义和公正，最终必能获得吉祥。

讼卦的应用广泛，可以指导我们在处理各种问题和挑战时，如何保持正义和公正，如何做出正确的决策和选择，如何达成共识和解决问题。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20250805_091954"
# timestamp = "20240225_222843"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [15]:
qlora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4608, bias=False)

In [16]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [17]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '周易卦象之一乾卦是由六个阳爻组成，象征着天，乾为“干”谐音“乾”，乾卦代表万物之天，象征宇宙间的一切阳性物质。在周易中，乾卦象征着刚健、强盛、筆直与坚定，代表万物生长过程中刚健 upward的精神力量，是卦象中最为活跃和充满活力的卦象之一。', 'content': '乾卦的卦辞“元、亨、利、贞”分别象征刚健、通行、有利、正固，意味着在乾卦中，即使面对极大的挑战，也要保持刚健和勇敢，才能达到最终的成功。乾卦的卦象由两个阳爻夹一个阴爻构成，象征刚中带柔，刚中有正，是卦象中最为稳定和坚固的卦象之一。\n乾卦的象征意义主要体现在“刚健”和“行”上，代表万事万物生长过程中最刚健、最活跃、最高涨的状态。在周易中，乾卦象征 career（事业）、 progression（晋升）、战争（军事）、婚丧（决断）等，都体现刚健、正统、高效的行态。在运势上，乾卦象征刚健、积极、'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250805_091954）：
[gMASK]sop 解释下乾卦是什么？ 在《周易》六十四卦之中，乾卦位居首位，象征着天，是纯阳之卦，由六个阳爻组成，代表至刚至健之象。乾卦卦辞为‘元、亨、利、贞’，分别寓意开始、通达、有利、正固，预示着大吉大利和吉祥如意。
《象辞》言：‘天行健，君子以自强不息。’意指天道运行刚健不息，君子应效法天道，不断自我奋发向上。乾卦象征着万物生成和发展的原始动力，代表刚强、进取、自律的精神状态，教导人应自强不息，修德立行，方能有所成就。
邵雍认为，乾卦象征刚健旺盛，具有极大的发展潜能，但需谨防刚强过度，反而带来反噬。得此卦者，事业顺利，名利双收，若能把握机会，善用天时，必可成就非凡。
乾卦强调‘行正道’的重要性，虽有大通之利，却必须持守中正，不可自满或骄纵。乾象征龙，亦象征德才兼备的君子，唯有刚健有为、德行昭昭者，方可飞龙在天，成就非凡事业。
在人生运势、事业、婚恋等各方面，乾卦均主顺利与成功，但同时也提醒需谨守谦慎，不急躁冒进。在传统解卦中，乾卦更是强调天道运行之理，教导人应顺应天行，自强不息，以刚健之德行，成就人生正道。


In [18]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '卦象为“上卦：乾，中卦：坤，下卦：震”，卦辞：“初吉，终凶，凶吉相济。”《象辞》曰：“先动则占优，被动则占凶；居庙堂之高者得志，居江湖之远者失意；足不户步，户不户言；吉凶相济，凶中带吉，凶中带凶，凶中带吉。”', 'content': '讼卦是周易中我最喜欢的卦之一，卦象中包含了“刚”与“柔”的冲突和对立，以及“顺”与“逆”的矛盾，象征着手足口的战争。虽然“初吉，终凶”意味着开始时顺利 but 最终则凶险，显示了卦象中“吉”与“凶”的 mutual influence 相互影响。\n从哲学角度看，讼卦强调在复杂情况下要谨慎行事，避免冲突，以达到“不战而胜”的最高境界。卦中的“刚”和“柔”分别代表刚强者与柔顺者，而“象”则表示这是君子之勇，即在柔中带刚，顺中带逆，用行动来证明自己的德行与智慧。\n在事业上，讼卦意味着初段顺利但终极凶险，需要在开始时顺利小心，避免过于自满或轻敌，而最终则需要谨慎应对，避免冲突。在人际关系中，讼卦则强调尊重和平，避免矛盾，以达到和平共处的最高境界。\n在决策上，讼卦提示要充分考虑利弊，避免刚过于刚，顺过于顺，需要灵活应变，以柔克刚。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250805_091954）：
[gMASK]sop 周易中的讼卦是什么卦象 讼卦是周易六十四卦之一，由下卦乾（天）和上卦坎（水）构成，象征争讼。坎为险，乾为健，刚健与险陷相互对立，因此容易产生纷争与矛盾。卦象显示天水相背，流向相反，象征事理乖舛，人际冲突加剧。
讼卦告诫人们，争讼虽非绝不可取，但终究不是上策，务必谨慎。卦辞「有孚，窒惕，中吉，终凶」意为虽有诚信，但仍需戒惧小心；中间阶段尚可吉利，但最终多凶险。此卦提醒人们，谋事之初应审慎规划，避免纠纷。
《象》曰「君子以做事谋始」，意味着君子应从源头处防止争讼的发生，强调预防胜于补救。《断易天机》亦指出，讼卦为刚与险相争之象，多不吉，宜退避求和。邵雍解卦为「天高水深，达远不亲」，强调慎谋退守，敬畏无凶。
从运势上看，讼卦提示事与愿违、不顺多阻，尤其要防小人陷害。在事业上宜退让求和，避免陷入诉讼。经商时应讲求诚信，公平互利，避免冲突。求名不利，应修养提升，待时而动。婚姻中虽不完美，但若彼此体

In [19]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦是《周易》六十四卦之一，由下卦坎（水）和上卦坤（地）构成，卦象为地中有水，象征军队与百姓的关系。坎卦象征险难与流动，坤卦象征顺承与包容，二者相叠，体现“寓兵于农”、“以农养兵”的哲学理念。师卦的核心思想在于用兵之道应慎重且出于正当理由，圣人不得已而为之，但一旦出兵则应师出有名，顺乎形势，以化解危机、化凶为吉。', 'content': '师卦的卦辞为：“地中有水，军队与百姓。先凶后吉，得配与ros。”卦辞寓意初始阶段可能发生冲突，但只要等待合适的时机，合理调动资源，就能达成和解与合作。 Ros是俄语中“幸运”的意思，此处表示经过努力后获得的好运。\n师卦在传统解释中代表军队、战争与征战。坎象征险，坤象征顺，象征军队虽处于危险之中，但只要耐心等待、顺应局势，就能最终成功。此卦强调以德服人、和平征讨的重要性，主张合理运用兵力，避免 fancyplay 冒险。\n师卦的运势体现为先凶后吉，必须等待合适的时机才能转折。事业上需要耐心等待，合理调整策略，方可成功。婚恋上需谨慎对待对方，不也可行。\n在事业上，师卦认为乐队练声调不和谐则难以协调，必须寻找合适的搭档，才能共同完成任务。在领导层中，此卦表示领导者虽有才能，但需节制且谨慎，避免与部下发生冲突。在征战中，象征军队与百姓的矛盾，需通过沟通与理解化解。\n在传统哲学中，师卦意味着“以德服人”的征讨策略，必须顺应局势、合理调兵。此卦强调“先凶后吉”，认为只有经过努力、等待时机、合理配合，才能最终成功。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250805_091954）：
[gMASK]sop 师卦是什么？ 师卦是《周易》六十四卦之一，由下卦坎（水）和上卦坤（地）构成，卦象为地中有水，象征军队与百姓的关系。坎卦象征险难与流动，坤卦象征顺承与包容，二者相叠，体现“寓兵于农”、“以农养兵”的哲学理念。师卦的核心思想在于用兵之道应慎重且出于正当理由，圣人不得已而为之，但一旦出兵则应师出有名，顺乎形势，以化解危机、化凶为吉。
《象辞》有言：“地中有水，师。君子以容民畜众。”这是对师卦深刻哲理的延伸，意味着君子应如大地容纳江河一般，包容民众，培养大众。在治理国家与统御军队的过程中，包容和教化民众是成功的基础

## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [20]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [21]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的第一卦,也是八卦中代表阳、强、积极、刚毅、行动力、创造力的卦象。乾卦是由两个阴爻夹一个阳爻构成,象征着天、强、积极、刚毅、行动力、创造力和正义。

乾卦的含义非常丰富,可以引申为领导、管理、创造、行动、发展等方面。在《易经》中,乾卦是八卦之首,代表着天地自然的力量,象征着一切事物的变化和发展。乾卦所代表的能量是积极向上的,是推动事物发展的主要力量。

乾卦的变化为天、地、风、雨、云、火、山、泽等。这些变化代表了不同的天气和自然现象,也反映了事物的发展和变化。乾卦的卦辞中提到了“天行健,君子以自强不息”,表达了积极向上、不断自强不息的精神。

乾卦是《易经》中最吉利的卦象之一,象征着宇宙的力量和生命的活力,启示人们积极向上、刚毅坚定、追求进步。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？
乾卦是八卦之一，也是八宫图说、易经、易学中的重要元素。乾卦代表天、代表刚强、代表行动、代表力量。乾卦是由两个阴爻夹一个阳爻构成，象征天。

乾卦的卦辞是“元、亨、利、永”，表示天长地久、行动顺利、事业成功、长久不变。

乾卦的六爻分别代表不同的含义：

初爻：阳爻，表示天、代表新、代表开始、代表希望。
二爻：阴爻，表示地、代表顺从、代表承载、代表承载。
三爻：阳爻，表示天、代表强、代表行动、代表力量。
四爻：阴爻，表示地、表示静止、代表坚守、代表静止。
五爻：阳爻，表示天、代表高、代表超越、代表高远。
六爻：阳爻，表示天、代表刚、代表行动、代表力量。

乾卦的含义非常丰富，它既象征着阳刚之力量，也象征着顺从和承载。它告诉我们，要积极行动，勇往直前，同时也要坚守正道，顺从自然，才能达到事业成功的目标。


In [22]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
"地水师卦"并不是一个标准的卦象，可能是您误将“师卦”误认为是“地水师卦”。在《易经》中，师卦是第6卦，由两个阴爻夹一个阳爻构成，象征军队或众人。师卦的含义广泛，包括教导、引导、军队等。

如果您想了解更多关于《易经》卦象的知识，欢迎您继续提问。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？
地水师卦是易经中的一个卦象，由两个卦相组合而成：地卦和风卦。地卦代表大地，风卦代表风。师卦是由地卦下乘风卦上而得，表示风从大地吹过，具有春风化雨、滋润万物的意义。

在易经的八卦中，师卦具有教育、引导、感化、治理等作用。它告诉我们，要善于利用风的力量，让大地充满生机和活力。同时，师卦也提醒我们要有包容和谦逊的态度，尊重自然、尊重他人，才能达到和谐共生的境界。

师卦的具体含义需要结合具体的情境来理解，同时还需要和其他卦象相互参照。只有这样，才能真正领悟易经的深层次哲学。


In [23]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
《周易》中的讼卦（干卦下，坤卦上）是八卦之一，它是由两个卦相组合而成，具有解决矛盾、平息纷争的含义。这个卦象是由两个卦象相互交换而得出的，分别是乾卦（天）和坤卦（地）。乾卦代表刚强、积极、进取、阳刚等特质，而坤卦则代表顺从、温顺、承载、阴柔等特质。当这两个特质相互结合时，代表了刚强与顺从的结合，积极与承载的结合，从而可以平息纷争，解决问题。

讼卦的出现，主要原因是阴阳双方不能和谐共处，矛盾纷争不断。解决讼卦的关键在于调整阴阳两极的关系，使它们能够相互依存、和谐共处。具体解决方法要根据卦象、卦辞和六爻爻辞来分析判断。通过调整阴阳两极的关系，使双方能够达成共识，从而平息纷争，解决问题。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么？
周易中的讼卦是八卦之一，它的卦象是由两个离字叠加而成，离为火，火在上，火下为水，水火相克。

讼卦象征着诉讼、争端、诉讼等纷争。它提醒人们在处理争端时要保持冷静、客观，遵循法度和道德原则，避免过度争斗和暴力解决。

同时，讼卦也预示着诉讼可能会带来一定的损失和消耗，因此建议在处理争端时要慎重、理智，以避免不必要的损失。

周易中的讼卦也启示人们要善于调整自己的心态和行为，避免争端和诉讼的发生，从而达到和谐共处、共同发展的目的。
